In [1]:
import os
import pandas as pd
import numpy as np
import json

from modules.detect import Detector
from modules.classify import Classifier
from modules.crop import crop_image

import torch
#ImageFromVideo.get_image('data/video/2023032319.mov')
import cv2

import time
import requests
# Initialize a class instance

In [2]:
human_detector = Detector()
#dumping_classifier = Classifier()


YOLOR  aab8e04 torch 1.12.1 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192.0MB)



Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 36905341 gradients


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


c:\Users\user\anaconda3\lib\site-packages\torch\_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:482.)
  return self._grad
c:\Users\user\anaconda3\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
def Capture():
    
    headers = {
    "Content-Type": "application/json"
    }
    url = "http://jungyoon.store:8080/tensor"   
    
    
    cap = cv2.VideoCapture(1)
    print("width:", cap.get(3), "height : ", cap.get(4))
    
    if not cap.isOpened():
        print("Camera open failed!")
        exit()
    
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    delay = round(500/fps)
    print("fps =" , fps, "delay = ", delay)
    out = cv2.VideoWriter('output.avi', fourcc, fps, (w,h))

    if not out.isOpened():
        print('File open failed!')
        cap.release()
        exit()
    
    start_time = time.time()
    
    detected = False
    
    while(True):
        ret, frame = cap.read()
        
        if ret:
            image = cv2.flip(frame, 1)
            
            if not detected:
                with torch.no_grad():
                    det = human_detector.detect(image)
            
                for i, (*xyxy, conf, cls) in enumerate(reversed(det)):
                    if cls == 0:
                        print("human detected")
                        detected = True
                        start_time = time.time()
                        print(start_time)
            
            if detected == True:
                out.write(image)
                                    
                current_time = time.time()
                elapsed_time = current_time - start_time
                if elapsed_time >= 2.0:
                    start_time = current_time
                    print("video saved and reset")
                    out.release()
                    out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))
                    
                    request = {'results' : "True"}
                    
                    data_json = json.dumps(request)
                    
                    r = requests.post(url, data=data_json, headers=headers)
                    
                    if r == 200:
                        print("post succeed", r.content.decode('utf-8'))
                    else:
                        print("post failed", r.status_code)
                    detected = False
            
            cv2.imshow("VIDEO", image)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [6]:
Capture()

width: 640.0 height :  480.0
fps = 30.0 delay =  17
human detected
1693287882.1614149
video saved and reset
human detected
1693287884.2264478
video saved and reset
human detected
1693287886.2871125
video saved and reset
human detected
1693287888.3762195
video saved and reset
human detected
1693287890.4856532
video saved and reset
human detected
1693287892.578293
video saved and reset
human detected
1693287894.6595874
video saved and reset
human detected
1693287896.7533405
video saved and reset
human detected
1693287898.8205068
video saved and reset
human detected
1693287900.8978467
video saved and reset
human detected
1693287902.977379
video saved and reset
human detected
1693287905.0426621
video saved and reset
human detected
1693287907.1222708
video saved and reset
human detected
1693287909.2137787
video saved and reset
human detected
1693287911.2727716
video saved and reset
human detected
1693287913.3948836
video saved and reset
human detected
1693287915.4752855
video saved and rese